In [1]:
from openai import OpenAI
from prompts import cheerful_tone_aug_sys_prompt
from functools import cache

import os
import json
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

### TODO:
- for each query generate: colloquial and annoyed versions
- save the newly generated queries alongside the original queries

In [3]:
# @cache
def generate_cheerful(queries: dict[str, list[str]]):
    res = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": cheerful_tone_aug_sys_prompt},
            {"role": "user", "content": json.dumps(queries, indent=4)},
        ],
        response_format={"type": "json_object"},
    )
    return res.choices[0].message.content


In [4]:
with open("queries.jsonl", 'r') as file:
    cheerful_queries = []
    for line in file:
        queries = json.loads(line)
        queries = {"customer_queries": queries["customer_queries"]}
        res = generate_cheerful(queries)
        res = json.loads(res)
        res.update({"tone": "cheerful"})
        cheerful_queries.append(res)

In [5]:
with open("queries.jsonl", 'a') as file:
    for query in cheerful_queries:
        json.dump(query, file, ensure_ascii=False)
        file.write('\n')